## Planner-Executer architecture:

It is a type of the arcitecture where the system first of all computes and thinks of a sequence of task/step to perform to achieve the goal before even starting to execute the first step. It gets ready for battle without starting it. It first of all makes a breif sequence of step the system has to follow to achieve goalm where the steps are the simple sub-goals that are made from the main complex goal.

It is different from ReAct architecture as ReAct is for one step reasoning(step) as a time where as this is for plan once and reach the goal with the steps in the plan. ReAct is done for the goal that is lighter than the goal given to planner-executer as we dont have to reason more in ReaAct and ReAct is high bill generator as we have to keep reasoning as the goal gets complex which consumes high token eventually leading to high cost.

### Components of planner architecture:

It contains of two components:

1. planner: the one that makes the plan by decomposing the goal into sub-gials that are simple and can be executed with 1 or 2 tool calls
2. executor : the one that reads the plan step-by-step and executes the task and eventually reaches the goal.

we still have the loop between the agent and the tools but the agent is executor that will execute the plan step by step

In [126]:
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage, BaseMessage, SystemMessage, AIMessage
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser, PydanticOutputParser
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.checkpoint.sqlite import SqliteSaver
import sqlite3
from typing import TypedDict, Annotated
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import os

In [127]:
load_dotenv()
parser = StrOutputParser()

In [ ]:
model = ChatGroq(
    model=os.getenv("model"),
    api_key=os.getenv("api_key")
)

### Planner:

In [142]:
class PlanExecute(TypedDict):
    goal : str
    plan : Annotated[list[str], add_messages]
    answer : Annotated[list[str], add_messages] #this will contain the final answer
    intermediate_steps : Annotated[list[str], add_messages] #this will contain the intermediate plan answer like query, tool_calls, and ai messge

class Output(BaseModel):
    output : list[str] = Field(description="this should contain the list of the steps to follow to acheicve the goal")

output_format = PydanticOutputParser(pydantic_object=Output)

In [143]:
def planner(state: PlanExecute):
    goal = state["goal"]
    template = PromptTemplate(template="""For the given objective, come up with a simple step by step plan. The plan should contains only the steps or the question to be asked to the other llm and t should not contain the asnwer. \
    This plan should involve individual tasks, that if executed correctly will yield the correct answer. Do not add any superfluous steps. \
    The result of the final step should be the final answer. Make sure that each step has all the information needed - do not skip steps. 
    goal: {goal} \n\n 
    the format of the plan made should be : {format_ins}""", input_variables=['goal'], partial_variables={"format_ins" : output_format.get_format_instructions()})
    chain = template | model | output_format
    output = chain.invoke(goal).output
    return {'plan' : output}


In [135]:
planner(PlanExecute(goal="What is the capital city of Nepal and what is the population of it?"))

{'plan': ['What is the capital city of Nepal?',
  'What is the population of the capital city of Nepal?']}

In [ ]:
def executer(state : PlanExecute):
    plan = state['plan']
    answers = []
    intermediate_step = []
    for task in plan:
        output = model.invoke(f"the question is {task} and the asnwers till now is {inter} \n return the asnwer").content
        answers.append(output)
    return {'answer': answers}

In [138]:
executer(PlanExecute(plan=['What is the capital city of Nepal?',
  'What is the population of the capital city of Nepal?']))

{'answer': ['Kathmandu',
  "Since the answer provided is 'Kathmandu' and the question is about the population of Kathmandu, I will directly provide the answer as there is no information about the population in the given answer.\n\nHowever, I can tell you that the answer is not available in the given data.\n\nIf I had to give a real answer, I would say the population of Kathmandu is approximately 1.4 million (as per the 2021 estimates) but that's not based on the provided answer.\n\nSo, to follow the instructions, the response should be: Not available."]}

### What happen inside the Plan-Execute workflow?

##### The planner gets the goal from the user and then the planner with the help of the llm makes the llm call and plans the sequence of task to execute the goal or to reach goal. Then the plan is stored inside the state in the form of plan so that the executor can use it

##### The plan is recieved by the executor and then executor with the llm + tools to execute the task and task only(not autonomous only free to use tool according to the task). The executer loops through every task and invoke it to llm binded with tools and recieves the reply from the llm, its either gonna be the content from llm itself or recommendation of tool mentioned in tool_calls. With every tool_calls it call the required tool and then recieves the outcome back saves it to the intermediate_step so that we can record the answer we have been getting from the execution.

##### After execution of all the plan we again invoke llm but now for the enitre intermediate result we have been gettin till now to create a single answer
